In [4]:
import pandas as pd
import networkx as nx
from collections import defaultdict, Counter
import numpy as np
from functions import*
import re
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

# Extract Data

In [7]:
academic     = pd.read_csv("data/academic.tsv",sep='\t')
advises      = pd.read_csv("data/advises.tsv",sep='\t')
degree_grant = pd.read_csv("data/degree_grant.tsv",sep='\t')
school       = pd.read_csv("data/school.tsv", sep='\t')
country      = pd.read_csv("data/country.tsv", sep='\t')
country2CN = pd.Series(country.country_name.values,index=country.country_id).to_dict()

degree1 = pd.read_csv("data/new_degree.tsv", sep='\t',keep_default_na=True)
degree = degree1.dropna(subset=["year"])

## Gives Full Name
full_name = academic.given_name + " " + academic.family_name
fam_name_D = pd.Series(academic.family_name.values,index=academic.academic_id).to_dict()
given_name_D = pd.Series(academic.given_name.values,index=academic.academic_id).to_dict()
academic["Full_Name"] = full_name
Fullname2ID = pd.Series(academic.academic_id.values, index=academic.Full_Name).to_dict()
full_name_D = pd.Series(full_name.values,index=academic.academic_id).to_dict()

academic2degree = pd.Series(degree1.degree_id.values,index=degree1.academic).to_dict()
degree2school   = pd.Series(degree_grant.school.values,index=degree_grant.degree).to_dict()
school2country  = pd.Series(school.country.values,index=school.school_id).to_dict()
school2SchName  = pd.Series(school.school_name.values,index=school.school_id).to_dict()

In [8]:
def ID2country(n1):
    degree_id  = academic2degree[n1]
    school_id  = degree2school[ degree_id ]
    country_id = school2country[ school_id ] 
    return country2CN[country_id]

def ID2school(n1):
    degree_id  = academic2degree[n1]
    school_id  = degree2school[ degree_id ]
    return school2SchName[school_id]

def ID2counID(n1):
    degree_id  = academic2degree[n1]
    school_id  = degree2school[ degree_id ]
    country_id = school2country[ school_id ] 
    return country_id

def ID2schID(n1):
    degree_id  = academic2degree[n1]
    school_id  = degree2school[ degree_id ]
    return school_id

In [10]:
adj_list = list(zip(advises["advisor"], advises["advisee"]))
G = nx.DiGraph()
G.add_edges_from(adj_list)
# Add Names
nx.set_node_attributes(G, full_name_D, 'full_name')
G_undirect = G.to_undirected()

medalists     = pd.read_csv("data/medalists.csv",sep=',')
medalist_IDs = []

for i in medalists.Winner.values:
    ID = Fullname2ID[i]
    if i == "Paul Cohen":
        ID = 6479
    if i == "Michael Freedman":
        ID = 1365
    if i == "Alan Baker":
        ID = 22765
    if ID == 230591:
        ID = 93772
    if ID == 245820:
        ID = 15779
    if ID == 211588:
        ID = 6488
    G.nodes()[ID]["medalist"] = 1
    medalist_IDs.append(ID)  

In [7]:
med_adj_list = []
for i in medalist_IDs:
    for j in medalist_IDs:
        if (i,j) in adj_list:
            med_adj_list.append((i,j))
# Gives nodes in subgraph
subgraph = defaultdict(int)
for i, a in enumerate(medalist_IDs):
    subgraph[a] = 1
    for j, b in enumerate(medalist_IDs[i+1:]):
        if nx.has_path(G_undirect,a,b):
            path_nodes = nx.shortest_path(G_undirect,a,b)
            for p in path_nodes:
                subgraph[p] = 1

In [8]:
subgraph_nodes = list(subgraph.keys())
K = G.subgraph(subgraph_nodes)
M = G.subgraph(medalist_IDs)

In [9]:
UNI_G  = defaultdict(int)
CTRY_G = defaultdict(int)
for n in G.nodes():
    try:
        UNI_G[ ID2school(n) ] += 1
    except KeyError:
        UNI_G[ "Other" ] += 1
    try:
        CTRY_G[ ID2country(n) ] += 1
    except KeyError:
        CTRY_G[ "Other" ] += 1

## Probabilities / Medalist

In [10]:
P_M  = nx.number_of_nodes(K) / nx.number_of_nodes(G) 
all_conditions = [] # Iteratively append different conditions

0.007589440925318205

In [13]:
thresh = 50
sorted_M = extract_hist(M,ID2school, thresh, group = False)
sorted_M = defaultdict(int, sorted_M)
ID2Elite = {}
for n in G.nodes():
    try:
        if sorted_M[ ID2school(n) ]: # in threshold
            ID2Elite[n] = "Elite"
        else:
            ID2Elite[n] = "Not Elite"
    except KeyError:
        ID2Elite[n] = "Not Elite"

In [14]:
UU = show_stats_K(ID2Elite, G)
all_conditions+= UU
all_conditions

Not Elite, 890, 203399, 0.004375636065074066
Power Ratio 0.5765426080960776
Elite, 900, 32455, 0.02773070405176398
Power Ratio 3.6538533371087945


[<__main__.condition at 0x1ed36041a48>, <__main__.condition at 0x1ed360414c8>]

In [15]:
def vary_thresh(G,thresh_list):
    stats = np.zeros((2,len(thresh_list)))
    
    for i, thresh in enumerate(thresh_list):
        sorted_M = extract_hist(M,ID2school, thresh, group = False)
        sorted_M = defaultdict(int, sorted_M)
        ID2Elite = {}
        for n in G.nodes():
            try:
                if sorted_M[ ID2school(n) ]: # in threshold
                    ID2Elite[n] = "Elite"
                else:
                    ID2Elite[n] = "Not Elite"
            except KeyError:
                ID2Elite[n] = "Not Elite"
                
        keys = extract_possible_keys(ID2Elite,G)
        for j,k in enumerate(keys):
            count_M, count_G, ratio = conditional_prob_K(k, ID2Elite,G)
            power_ratio = ratio/P_M
            stats[j,i] = power_ratio
    return stats

probabilities = vary_thresh(G,list(range(1,50)))

### Ethnicity

In [16]:
WIKI_FULL   = pd.read_pickle("wiki_fullname.pkl")

In [17]:
Wiki_General = defaultdict(int)
Wiki_Specific = defaultdict(int)
for i, row in WIKI_FULL.iterrows():
    n = row.race
    general_region = n.split(",")[0] 
    specif_region  = n.split(",")[-1] 
    if specif_region == "Jewish":
        n = (row.iloc[4:].sort_values(ascending = False).index[1] )
        specif_region  = n.split(",")[-1]    
    if specif_region == "Muslim":
        specif_region = "Arabic"
    elif specif_region == "Hispanic":
        specif_region = "Spanish"
    elif specif_region == "British":
        specif_region = "Anglo"
    
    Wiki_General[ row.academic_id ]  = general_region
    Wiki_Specific[ row.academic_id ] = specif_region


EastEuropean
Muslim
French
British
British
Germanic
British
British
British
British
EastEuropean
British
British
Germanic
British
Nordic
French
British
British
British
British
Germanic
EastEuropean
Germanic
British
British
British
British
British
Germanic
British
British
British
Germanic
British
British
Germanic
EastEuropean
Germanic
British
British
British
British
Muslim
Germanic
Muslim
Nordic
British
Hispanic
British
British
British
British
Germanic
Germanic
British
EastEuropean
British
British
British
Italian
British
British
EastEuropean
French
EastEuropean
British
EastEuropean
Germanic
British
British
French
British
Muslim
IndianSubContinent
French
British
British
British
Germanic
British
British
Germanic
British
EastEuropean
EastEuropean
British
British
French
British
British
British
EastEuropean
Muslim
EastEuropean
Germanic
IndianSubContinent
Muslim
EastEuropean
British
British
British
British
British
Muslim
Germanic
British
French
French
Germanic
British
British
British
French
B

Germanic
French
British
British
British
Germanic
EastEuropean
EastEuropean
EastEuropean
British
EastEuropean
British
EastEuropean
French
Germanic
British
Africans
British
Germanic
British
Muslim
British
British
Germanic
Germanic
Italian
British
British
Germanic
Germanic
French
British
EastEuropean
British
British
British
Germanic
British
British
British
Germanic
British
French
British
British
Hispanic
Germanic
EastEuropean
British
British
Germanic
Germanic
Germanic
British
British
British
British
British
British
EastEuropean
British
EastEuropean
British
French
French
British
British
EastEuropean
British
EastEuropean
Germanic
British
EastEuropean
British
British
British
British
Germanic
British
British
Germanic
EastEuropean
EastEuropean
Germanic
British
British
British
British
British
British
British
Germanic
Germanic
British
EastEuropean
British
British
British
EastEuropean
Africans
British
British
French
British
Muslim
Muslim
French
British
British
British
EastEuropean
EastEuropean
Br

Germanic
Muslim
Germanic
Germanic
British
EastEuropean
British
EastEuropean
Germanic
Nordic
Germanic
IndianSubContinent
Germanic
British
British
French
Germanic
Germanic
French
Germanic
Germanic
British
British
Germanic
Germanic
British
Germanic
EastEuropean
Germanic
Germanic
Germanic
Germanic
French
Germanic
Germanic
Germanic
Germanic
Germanic
EastEuropean
Nordic
Germanic
Germanic
IndianSubContinent
Germanic
British
British
British
EastEuropean
French
British
Germanic
Germanic
Muslim
EastEuropean
Germanic
Germanic
British
EastEuropean
British
Germanic
British
French
British
Germanic
British
Nordic
British
Germanic
British
French
British
Hispanic
Germanic
Germanic
EastEuropean
Hispanic
British
British
British
Germanic
British
Germanic
EastEuropean
British
British
Germanic
British
Germanic
EastEuropean
EastAsian
Germanic
British
British
EastEuropean
British
Hispanic
EastEuropean
British
British
EastEuropean
EastEuropean
Nordic
French
Hispanic
Germanic
Muslim
Germanic
Germanic
Germanic
G

British
British
British
French
British
EastEuropean
Nordic
British
British
British
British
British
Germanic
British
Germanic
British
British
British
British
Germanic
Germanic
Germanic
Germanic
British
Germanic
EastEuropean
EastEuropean
British
Hispanic
EastEuropean
Germanic
EastEuropean
British
British
Germanic
British
Muslim
British
British
British
Germanic
British
EastEuropean
Germanic
Germanic
British
EastEuropean
Germanic
Nordic
Muslim
Germanic
Germanic
Germanic
British
Germanic
British
Germanic
Germanic
Germanic
Germanic
British
Germanic
British
British
Germanic
British
British
Hispanic
Germanic
British
Germanic
British
French
Germanic
Germanic
Germanic
Italian
French
British
Germanic
British
Germanic
British
British
EastEuropean
EastAsian
Germanic
Germanic
French
Germanic
Germanic
Germanic
Germanic
Germanic
IndianSubContinent
British
Germanic
Germanic
Germanic
Germanic
British
Germanic
Nordic
Germanic
Germanic
British
French
Germanic
Germanic
Germanic
Germanic
Germanic
British
Ge

British
Italian
British
EastEuropean
EastEuropean
Germanic
Hispanic
British
Muslim
Italian
Germanic
British
British
British
British
British
British
British
British
British
EastEuropean
Germanic
British
EastEuropean
Germanic
British
EastAsian
EastEuropean
French
British
British
British
British
British
EastEuropean
Nordic
IndianSubContinent
Germanic
Germanic
British
EastEuropean
Germanic
British
EastEuropean
British
British
British
Germanic
British
British
British
British
EastEuropean
British
EastEuropean
British
British
British
British
British
British
British
British
Germanic
Nordic
French
British
Germanic
EastEuropean
British
Hispanic
Germanic
Germanic
Germanic
British
Muslim
British
Hispanic
EastEuropean
British
British
British
Germanic
British
British
British
French
British
British
EastEuropean
EastEuropean
British
British
EastEuropean
British
French
British
EastEuropean
British
Germanic
French
Germanic
Germanic
Germanic
British
EastEuropean
British
Muslim
Germanic
EastEuropean
Germa

...

Germanic
EastEuropean
British
Muslim
Germanic
Germanic
French
EastEuropean
EastEuropean
British
EastEuropean
Germanic
Germanic
Germanic
Muslim
Germanic
IndianSubContinent
EastEuropean
Italian
British
British
British
Germanic
British
British
Germanic
IndianSubContinent
Germanic
IndianSubContinent
Hispanic
Germanic
Germanic
French
Germanic
British
Muslim
EastEuropean
British
Germanic
British
British
British
British
British
Germanic
Germanic
British
Germanic
British
British
French
British
Germanic
British
EastEuropean
Germanic
EastEuropean
EastEuropean
British
Germanic
French
Germanic
British
EastEuropean
British
Germanic
British
EastEuropean
EastEuropean
EastEuropean
Germanic
British
Germanic
British
Germanic
British
Muslim
EastEuropean
Germanic
EastEuropean
Hispanic
British
EastEuropean
British
Germanic
Germanic
British
EastEuropean
Hispanic
Germanic
British
Germanic
Germanic
Japanese
British
Germanic
British
Germanic
Hispanic
Germanic
Germanic
Germanic
Muslim
EastEuropean
Germanic
Germ

British
British
British
Germanic
British
Germanic
British
French
Nordic
Germanic
EastEuropean
Muslim
Germanic
British
British
British
British
British
Germanic
British
British
British
Germanic
British
Germanic
British
British
Germanic
British
British
Germanic
Muslim
Germanic
British
British
British
French
Italian
EastEuropean
British
Germanic
British
EastEuropean
Muslim
EastEuropean
Germanic
British
Japanese
British
British
EastEuropean
Germanic
British
British
British
Germanic
Muslim
British
Germanic
Germanic
British
Germanic
Germanic
British
Germanic
Germanic
Germanic
British
British
British
British
EastEuropean
British
British
British
British
Nordic
British
British
British
Italian
Italian
British
EastEuropean
Germanic
French
British
British
British
Germanic
Germanic
Germanic
Germanic
British
British
EastEuropean
Muslim
British
British
Germanic
Germanic
British
French
British
British
British
British
British
British
EastEuropean
Italian
British
Italian
British
Germanic
British
British


British
EastEuropean
Germanic
British
British
British
EastEuropean
British
Germanic
Germanic
IndianSubContinent
Germanic
British
Nordic
Germanic
EastEuropean
Germanic
British
Germanic
French
Germanic
Germanic
Germanic
British
French
Germanic
British
EastAsian
Hispanic
EastEuropean
British
British
Muslim
British
EastEuropean
Germanic
Italian
British
British
Germanic
British
Germanic
Germanic
British
EastEuropean
Germanic
Japanese
Germanic
Germanic
British
British
British
British
EastEuropean
Muslim
British
British
British
Germanic
EastEuropean
British
British
Italian
British
Germanic
British
British
British
British
British
Germanic
French
British
British
Muslim
Germanic
Germanic
Germanic
British
French
British
EastEuropean
British
British
British
British
British
British
British
Germanic
EastEuropean
IndianSubContinent
British
Germanic
EastEuropean
French
Africans
EastEuropean
British
British
British
Italian
British
British
Germanic
Germanic
EastEuropean
IndianSubContinent
Hispanic
Germa

British
Germanic
Muslim
EastEuropean
EastEuropean
British
British
British
Germanic
British
French
British
British
British
EastEuropean
French
French
Germanic
Africans
British
British
British
British
Muslim
British
French
Germanic
IndianSubContinent
Germanic
Germanic
Germanic
Germanic
Germanic
Germanic
Germanic
British
Germanic
Muslim
Germanic
British
Muslim
British
British
Muslim
Africans
Germanic
EastEuropean
Muslim
Germanic
EastEuropean
British
British
British
Italian
EastEuropean
EastEuropean
British
Germanic
Germanic
British
British
Muslim
British
Germanic
French
Muslim
British
British
Germanic
British
British
IndianSubContinent
Germanic
Germanic
French
Germanic
Germanic
British
British
Germanic
EastEuropean
Germanic
British
British
French
EastEuropean
Germanic
French
Germanic
British
Muslim
Germanic
British
British
British
Germanic
Germanic
Germanic
British
British
EastEuropean
Germanic
British
British
French
British
British
Germanic
EastEuropean
British
British
Muslim
Hispanic
Br

Nordic
Muslim
Muslim
EastEuropean
British
Germanic
Italian
French
EastEuropean
British
British
Germanic
British
Germanic
Italian
British
British
British
British
Germanic
British
Germanic
British
British
British
British
Germanic
British
Muslim
Muslim
British
British
British
Germanic
EastEuropean
British
Germanic
EastEuropean
British
Germanic
EastEuropean
British
Muslim
British
Germanic
British
British
British
Germanic
British
British
French
Muslim
British
Germanic
British
British
British
British
British
Germanic
French
British
British
EastEuropean
Africans
EastEuropean
Germanic
Muslim
EastEuropean
British
EastEuropean
British
British
French
Germanic
British
British
British
British
EastEuropean
EastEuropean
EastEuropean
British
British
British
British
EastEuropean
Germanic
French
British
British
Germanic
French
British
Germanic
Africans
EastAsian
EastEuropean
Muslim
IndianSubContinent
EastEuropean
Germanic
EastEuropean
French
British
British
Germanic
Hispanic
French
Italian
Muslim
EastEu

In [18]:
GEN = show_stats_K(Wiki_General, G)
SPE = show_stats_K(Wiki_Specific, G)

GreaterEuropean, 1573, 174957, 0.008990780591802557
Power Ratio 1.1846433327927377
Asian, 163, 45884, 0.003552436579199721
Power Ratio 0.4680761882405425
GreaterAfrican, 54, 15013, 0.0035968827016585624
Power Ratio 0.47393249872456905


### Second Order

In [20]:
compoundSpecific = defaultdict(int)
compoundGeneral  = defaultdict(int)
for n in G.nodes():
    if Wiki_Specific[n] == "IndianSubContinent":
        compoundGeneral[n]  = ID2Elite[n] + "|" + Wiki_General[n]
        compoundSpecific[n] = ID2Elite[n] + "|" + "Indian"
    else:
        compoundGeneral[n]  = ID2Elite[n] + "|" + Wiki_General[n]
        compoundSpecific[n] = ID2Elite[n] + "|" + Wiki_Specific[n]

In [21]:
SecOrder_GEN = show_stats_K(compoundGeneral,G)

Not Elite|GreaterEuropean, 755, 148717, 0.005076756524136447
Power Ratio 0.6689236498568031
Elite|GreaterEuropean, 818, 26240, 0.031173780487804878
Power Ratio 4.107520012944543
Not Elite|Asian, 102, 41090, 0.0024823558043319542
Power Ratio 0.327080193226206
Not Elite|GreaterAfrican, 33, 13592, 0.002427898763978811
Power Ratio 0.3199048240667366
Elite|Asian, 61, 4794, 0.012724238631622862
Power Ratio 1.6765712727501556
Elite|GreaterAfrican, 21, 1421, 0.014778325123152709
Power Ratio 1.9472218399977983


In [22]:
SecOrder_SPE = show_stats_K(compoundSpecific,G)

Not Elite|Anglo, 249, 66525, 0.0037429537767756484
Power Ratio 0.4931791173562256
Not Elite|French, 106, 11670, 0.00908311910882605
Power Ratio 1.1968100415045022
Elite|French, 141, 2874, 0.049060542797494784
Power Ratio 6.464315788245997
Not Elite|Spanish, 81, 13917, 0.005820219874973055
Power Ratio 0.7668838761965893
Not Elite|Italian, 83, 10278, 0.00807550107024713
Power Ratio 1.0640442622469646
Elite|Anglo, 367, 13719, 0.02675122093447044
Power Ratio 3.5247946716640177
Not Elite|EastAsian, 56, 25980, 0.002155504234026174
Power Ratio 0.28401357296760293
Not Elite|Indian, 22, 13036, 0.0016876342436330162
Power Ratio 0.22236608206582203
Not Elite|Germanic, 108, 21138, 0.00510928186204939
Power Ratio 0.6732092537942999
Not Elite|Africans, 9, 3818, 0.002357255107386066
Power Ratio 0.31059667379744876
Not Elite|Arabic, 26, 11025, 0.00235827664399093
Power Ratio 0.31073127351499263
Elite|Nordic, 23, 858, 0.026806526806526808
Power Ratio 3.5320818845958515
Not Elite|EastEuropean, 90, 17316

In [23]:
df = pd.DataFrame(columns = ["Name", "cM", "cG", "ratio", "PR"])

for n in SecOrder_SPE:
    S = pd.Series([n.name,n.cM,n.cG,n.ratio,n.PR] ,  index=["Name", "cM", "cG", "ratio", "PR"])
    df = df.append(S, ignore_index=True)
    
new = df["Name"].str.split("|", n =1, expand = True) 
df["Elitism"]= new[0] 
# making separate last name column from new data frame 
df["Name"]= new[1] 
sorted_df = df.sort_values(["Elitism","Name"])

In [24]:
%matplotlib qt

In [29]:
plt.figure(figsize = (14,8))
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
heat_df = sorted_df[["Elitism", "Name","PR"]]
pivot = pd.pivot_table(heat_df,values=["PR"],index=['Name'], columns=['Elitism'])
pivot = pivot.sort_values(("PR","Elite"), ascending=False)

ax = sns.heatmap(pivot,cmap="Reds",annot=True, fmt=".3f", linewidths=.5,annot_kws={"size":20})
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.xticks([0.5,1.5], ["Elite", "Not Elite"])
plt.xlabel("Institution (Top {})".format(thresh), fontsize = 20)
plt.ylabel("Name Ethnicity", fontsize = 20)

Text(131.72222222222223, 0.5, 'Name Ethnicity')

### General

In [30]:
df_gen = pd.DataFrame(columns = ["Name", "cM", "cG", "ratio", "PR"])

for n in SecOrder_GEN:
    S = pd.Series([n.name,n.cM,n.cG,n.ratio,n.PR] ,  index=["Name", "cM", "cG", "ratio", "PR"])
    df_gen = df_gen.append(S, ignore_index=True)
    
new = df_gen["Name"].str.split("|", n =1, expand = True) 
df_gen["Elitism"]= new[0] 
# making separate last name column from new data frame 
df_gen["Name"]= new[1] 
# df_gen = df_gen[df_gen.Elitism == "Elite"]
sorted_gen = df_gen.sort_values(["Elitism","Name"])

In [32]:
plt.figure(figsize = (14,10))
matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
heat_gen = sorted_gen[["Elitism", "Name","PR"]]
pivot_gen = pd.pivot_table(heat_gen,values=["PR"],index=['Name'], columns=['Elitism'])
pivot_gen = pivot_gen.sort_values(("PR","Elite"), ascending=False)

ax = sns.heatmap(pivot_gen,cmap="Reds",annot=True, fmt=".3f", linewidths=.5,annot_kws={"size":26})
plt.xticks([0.5, 1.5], ["Elite", "Not Elite"])
plt.yticks([0.3,1.3,2.3], ["European" , "Asian" , "African",],rotation=90,)
plt.xlabel("Institutions".format(thresh), fontsize = 24)
plt.ylabel(" ", fontsize = 2)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

(2.8, -0.2)